**7.2. Buidling AI Agent with Pydantic AI**

**AI Agent:**

An AI agent is a software program that uses artificial intelligence to autonomously perceive its environment, reason, plan, and act to achieve specific goals, often with minimal human intervention


**Tools**

In AI agent systems, a tool is a predefined function or external service that an agent can call to perform specific actions or access information beyond its core capabilities. Tools bridge the gap between an agent's reasoning and real-world action, allowing it to retrieve data, execute code, interact with APIs, or perform other specialized tasks needed to fulfill a user's request.

In [24]:
!pip install -q -U requests pydantic-ai "pydantic-ai-slim[groq]"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


**NOTE:** Get your Groq API key & Open Weather Map API Key

Open weather map api: https://home.openweathermap.org/api_keys

In [ ]:
import nest_asyncio

# event loop, important in jupiter nootbook to use exsiting one
nest_asyncio.apply()

In [6]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "your_openweathermap_api_key"

def find_weather(city: str) -> dict:
    """This function returns the current weather forecast for the given city"""
    units = "metric"
    params = {
      'q': city,
      'appid': API_KEY,
      'units': units
    }
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    return result

In [7]:
output = find_weather("London")
print(output)

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 15.24, 'feels_like': 15.04, 'temp_min': 15.24, 'temp_max': 15.24, 'pressure': 1020, 'humidity': 85, 'sea_level': 1020, 'grnd_level': 1016}, 'visibility': 10000, 'wind': {'speed': 3.44, 'deg': 136, 'gust': 8.46}, 'clouds': {'all': 100}, 'dt': 1757146298, 'sys': {'country': 'GB', 'sunrise': 1757136079, 'sunset': 1757183808}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


In [15]:
print(output["name"])
print(output["weather"][0]["description"].capitalize())
print(output["main"]["temp"])

London
Overcast clouds
15.24


**Pydantic AI Agent**

In [8]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [9]:
os.environ["GROQ_API_KEY"] = "your_groq_api_key"

In [10]:
# Define the output schema for the tool
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [13]:
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperaure=0.2),
    output_type=str,
    system_prompt=("You are a helpful weather assistant. Use the 'get_weather_forecast' tool" 
                   "to find current weather conditions for any city. Provide clean and friendly answers")
)

In [16]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "your_openweathermap_api_key"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(url, params=params)
    result = response.json()

    return WeatherForecast(
        location=result["name"],
        description=result["weather"][0]["description"].capitalize(),
        temperature_celsius=result["main"]["temp"]
    )

In [20]:
agent_response = weather_agent.run_sync("What is the weather in Chennai")
print(agent_response.output)

It appears that the current weather forecast for Chennai is mostly sunny with scattered clouds and a temperature of around 32°C.


In [21]:
question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)

🌤️ Ask about the weather:  London



📍 Forecast: The current weather forecast for London is mostly overcast with a temperature of around 17.57°C.


In [22]:
"""
Weather Assistant using Pydantic-AI and OpenWeatherMap API
This script creates a conversational agent that can respond to weather-related queries
using the OpenWeatherMap API and a Groq-hosted LLaMA model.
"""

# Allow nested async loops (useful for Jupyter or async environments)
import nest_asyncio
nest_asyncio.apply()

# Standard libraries
import os
import requests

# Pydantic model for structured data
from pydantic import BaseModel

# Core AI libraries from pydantic_ai
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

os.environ["GROQ_API_KEY"] = "your_groq_api_key" 

# 1. Define the output schema of the tool using Pydantic
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

# 2. Create the AI agent using Groq’s LLaMA 3 model
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=(
        "You are a helpful weather assistant. Use the 'get_weather_forecast' tool to "
        "find current weather conditions for any city. Provide clean and friendly answers."
    )
)

# 3. Register a tool with the agent to fetch real-time weather using OpenWeatherMap API
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Tool: get_weather_forecast
    Description: Fetches current weather for a city using the OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    
    # Replace this with your own API key for production use
    api_key = "your_openweathermap_api_key"
    
    # Query parameters
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    # Send request to weather API
    res = requests.get(url, params=params).json()

    # Return the formatted weather information
    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

# 4. Run continuous user interaction loop
if __name__ == "__main__":
    print("🌦️  Weather Assistant is ready! Type 'exit' to quit.")
    print("‒" * 50)
    while True:
        question = input("🌤️ Ask about the weather: ").strip()
        if question.lower() in {"exit", "quit", ""}:
            print("\n👋 Exiting weather assistant. Have a nice day!")
            break

        try:
            result = weather_agent.run_sync(question)
            print("\n📍 Forecast:", result.output)
        except Exception as e:
            print("⚠️ Error:", str(e))

        print("‒" * 50)

🌦️  Weather Assistant is ready! Type 'exit' to quit.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒


🌤️ Ask about the weather:  Chennai



📍 Forecast: The current weather in Chennai is scattered clouds with a temperature of 30.39°C.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒


🌤️ Ask about the weather:  Paris



📍 Forecast: It's currently 17.19°C in Paris with broken clouds.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒


🌤️ Ask about the weather:  exit



👋 Exiting weather assistant. Have a nice day!
